Step 2: Text Analysis with NLP
Text Preprocessing:



1.   Tokenize the text data.
2.   Remove stop words, punctuation, and apply stemming/lemmatization.
3.    Convert text to lowercase.





In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from time import time
import re
import inflect
import unicodedata
from nltk.tokenize import word_tokenize


# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from google.colab import drive

drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
miao_feed_all_docs_dropped = pd.read_csv('/content/drive/MyDrive/nlp/miao_merged_NER_dataset_clean.csv')
news_feed_all_docs_dropped = pd.read_csv('/content/drive/MyDrive/nlp/news_merged_NER_dataset_clean.csv')

# miao_feed_all_docs_dropped = pd.read_csv('/content/drive/MyDrive/Data/miao_feed_all_docs_dropped.csv')
# news_feed_all_docs_dropped = pd.read_csv('/content/drive/MyDrive/Data/news_feed_all_docs_dropped.csv')

In [ ]:
miao_feed_all_docs_dropped.head()

,content,names,clubs,fee
0,AC Milan head coach Stefano Pioli says Rafael ...,"Stefano Pioli, Rafael Leao, Leao, Olivier Giro...","AC Milan, Champions League, Stadio Diego Arman...",NaN
1,Behdad Eghbali and Todd Boehly - Todd Boehly a...,"Behdad Eghbali, Todd Boehly - Todd Boehly, Tod...","Chelsea, Chelsea, Real Madrid, Chelsea, Chelse...",£600 million
2,"Love them or loathe them, agents are integral ...","Jonathan Barnett, Barnett, Barnett, Tottenham ...","FIFA, CAA Stellar, Rafaela Pimenta, FIFA, FIFA...","20 per cent, 8million, 9.9, 10 per cent, three..."
3,"If nothing else, Leeds United’s regression tow...","Leeds United’s, Simon Grayson’s, Robert Snodgr...","Liverpool, the Premier League, EFL, the Premie...","30million, 37.3, 17million"
4,"Two months ago, when the Glazer family’s plan ...","Glazer, Jassim, Jim Ratcliffe, Glazer, Jassim ...","Manchester United, the Manchester United Suppo...",20 league


In [ ]:
def clean_article(article):
    # Initialize the inflect engine
    p = inflect.engine()

    # Dictionary mapping currency symbols to words
    currency_map = {
        '\u0024': 'dollar',    # $
        '\u00A3': 'pound',     # £
        '\u00A5': 'yen',       # ¥
        '\u20AC': 'euro',      # €
        '\u20B9': 'rupee',     # ₹
        '\u20A1': 'colon',     # ₡
        '\u20A2': 'cruzeiro',  # ₢
        '\u20A3': 'french_franc', # ₣
        '\u20A4': 'lira',      # ₤
        '\u20A5': 'mill',      # ₥
        '\u20A6': 'naira',     # ₦
        '\u20A7': 'peseta',    # ₧
        '\u20A8': 'rupee',     # ₨
        '\u20A9': 'won',       # ₩
        '\u20AA': 'shekel',    # ₪
        '\u20AB': 'dong',      # ₫
        '\u20AC': 'euro',      # €
        '\u20AD': 'kip',       # ₭
        '\u20AE': 'tugrik',    # ₮
        '\u20AF': 'drachma',   # ₯
        '\u20B0': 'german_penny', # ₰
        '\u20B1': 'peso',      # ₱
        '\u20B2': 'guarani',   # ₲
        '\u20B3': 'austral',   # ₳
        '\u20B4': 'hryvnia',   # ₴
        '\u20B5': 'cedi',      # ₵
    }

    # Replace currency symbols with their corresponding words
    for symbol, word in currency_map.items():
        article = re.sub(symbol, word, article)

    # Replace numbers with words
    try:
        article = re.sub(r'\d+', lambda x: p.number_to_words(x.group()), article)
    except inflect.NumOutOfRangeError: # Changed to inflect.NumOutOfRangeError
        # Handle the error, e.g., replace with a placeholder
        article = re.sub(r'\d+', ' large_number ', article)

    # Remove URLs
    article = re.sub(r'https?://[^\s]+', 'url', article)

    # Remove special characters (except for standard punctuation)
    article = re.sub(r'[^\w\s.,;\'"-]', '', article)

    # Remove extra spaces, tabs, newlines, and carriage returns
    article = re.sub(r'\s+', ' ', article).strip()

    # Remove accented characters
    article = unicodedata.normalize('NFKD', article).encode('ASCII', 'ignore').decode('utf-8')

    # Tokenization
    tokens = word_tokenize(article)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]

    # Remove single letters (usually noise in articles)
    tokens = [token for token in tokens if len(token) > 1]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Rejoin tokens into a cleaned article
    cleaned_article = ' '.join(tokens)

    return cleaned_article

In [ ]:
# Initialize the inflect engine
p = inflect.engine()

def convert_all_numbers_to_words(text):
    # Pattern to match all numbers
    pattern = re.compile(r'\b\d+\b')

    # Function to replace matched pattern
    def replace(match):
        number = match.group(0)
        return p.number_to_words(number)

    # Replace numbers with words
    return pattern.sub(replace, text)

In [ ]:
# Assuming miao_feed_all_docs_dropped is your DataFrame
miao_feed_all_docs_content = miao_feed_all_docs_dropped['content']

# Define stop words and punctuation
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)
lemmatizer = WordNetLemmatizer()

# Tokenize, remove stopwords, punctuation and lemmatize
def process_text(text):
    text = convert_all_numbers_to_words(text)
    tokens = nltk.word_tokenize(text)
    processed_tokens = [
        lemmatizer.lemmatize(token.lower())
        for token in tokens
        if token.lower() not in stop_words and token not in punctuation
        ]
    return processed_tokens

# Measure execution time
start_time = time()

# Apply the process_text function to each entry in the 'content' column
miao_feed_all_docs_dropped['miao_feed_all_docs_processed_tokens'] = miao_feed_all_docs_content.apply(process_text)

# Measure end time and calculate duration
end_time = time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

Execution time: 658.6090047359467 seconds


In [ ]:
# Assuming miao_feed_all_docs_dropped is your DataFrame
miao_feed_all_docs_content = miao_feed_all_docs_dropped['content']

# Measure execution time
start_time = time()

# Apply the process_text function to each entry in the 'content' column
miao_feed_all_docs_dropped['miao_feed_all_docs_processed_tokens'] = miao_feed_all_docs_content.apply(clean_article)

# Measure end time and calculate duration
end_time = time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

Execution time: 686.9015934467316 seconds


In [ ]:
miao_feed_all_docs_dropped.head(2)

,content,names,clubs,fee,miao_feed_all_docs_processed_tokens
0,AC Milan head coach Stefano Pioli says Rafael ...,"Stefano Pioli, Rafael Leao, Leao, Olivier Giro...","AC Milan, Champions League, Stadio Diego Arman...",NaN,AC Milan head coach Stefano Pioli say Rafael L...
1,Behdad Eghbali and Todd Boehly - Todd Boehly a...,"Behdad Eghbali, Todd Boehly - Todd Boehly, Tod...","Chelsea, Chelsea, Real Madrid, Chelsea, Chelse...",£600 million,Behdad Eghbali Todd Boehly Todd Boehly Co nowh...


In [ ]:
#miao_feed_all_docs_dropped.to_csv('/content/drive/MyDrive/nlp data/Data/miao_feed_all_docs_dropped_lemmatize.csv', index=False)

miao_feed_all_docs_dropped.to_csv('/content/drive/MyDrive/nlp/miao_feed_all_docs_dropped_lemmatize.csv', index=False)

In [ ]:
#Drop keyword column
miao_feed_all_docs_dropped_clean = miao_feed_all_docs_dropped.drop(columns=['content'])

#Rename lemmatized column to token for uniformity
miao_feed_all_docs_dropped_clean.rename(columns={'miao_feed_all_docs_processed_tokens': 'token'}, inplace=True)

In [ ]:
miao_feed_all_docs_dropped_clean.head(2)

,names,clubs,fee,token
0,"Stefano Pioli, Rafael Leao, Leao, Olivier Giro...","AC Milan, Champions League, Stadio Diego Arman...",NaN,AC Milan head coach Stefano Pioli say Rafael L...
1,"Behdad Eghbali, Todd Boehly - Todd Boehly, Tod...","Chelsea, Chelsea, Real Madrid, Chelsea, Chelse...",£600 million,Behdad Eghbali Todd Boehly Todd Boehly Co nowh...


In [ ]:
# Measure execution time
start_time = time()

# Assuming miao_feed_all_docs_dropped is your DataFrame
news_feed_all_docs_content = news_feed_all_docs_dropped['content']

# Apply the process_text function to each entry in the 'content' column
news_feed_all_docs_dropped['news_feed_all_docs_processed_tokens'] = news_feed_all_docs_content.apply(clean_article)

# Measure end time and calculate duration
end_time = time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

Execution time: 832.1204400062561 seconds


In [ ]:
news_feed_all_docs_dropped.to_csv('/content/drive/MyDrive/nlp/news_feed_all_docs_dropped_lemmatize.csv', index=False)

In [ ]:
news_feed_all_docs_dropped.head(2)

,content,names,clubs,fee,news_feed_all_docs_processed_tokens
0,The top 10 most valuable Premier League goalke...,"Alisson, Alisson, Ederson, Arijanet Muric - Bu...","Premier League, Ederson and AlissonLiverpool, ...","20m/£17.3, 20m/£17.3, 20, 50m/£43.4, 43.8, 60m...",top ten valuable Premier League goalkeeper rev...
1,The England striker was involved in the crash ...,"Marcus Rashford, Rashford, HTML5, Maintenance,...","Manchester United, Sun, United, Rolls-Royce, U...","700,000",England striker involved crash grandmother yes...


In [ ]:
#Drop keyword column
news_feed_all_docs_dropped_clean = news_feed_all_docs_dropped.drop(columns=['content'])

#Rename lemmatized column to token for uniformity
news_feed_all_docs_dropped_clean.rename(columns={'news_feed_all_docs_processed_tokens': 'token'}, inplace=True)

In [ ]:
news_feed_all_docs_dropped_clean.head(2)

,names,clubs,fee,token
0,"Alisson, Alisson, Ederson, Arijanet Muric - Bu...","Premier League, Ederson and AlissonLiverpool, ...","20m/£17.3, 20m/£17.3, 20, 50m/£43.4, 43.8, 60m...",top ten valuable Premier League goalkeeper rev...
1,"Marcus Rashford, Rashford, HTML5, Maintenance,...","Manchester United, Sun, United, Rolls-Royce, U...","700,000",England striker involved crash grandmother yes...


In [ ]:
#Combine both news files into one
import pandas as pd

# Concatenate the DataFrames vertically
all_feed_all_docs_clean = pd.concat([news_feed_all_docs_dropped_clean, miao_feed_all_docs_dropped_clean], ignore_index=True)

# Print the concatenated DataFrame
all_feed_all_docs_clean.head()

,names,clubs,fee,token
0,"Alisson, Alisson, Ederson, Arijanet Muric - Bu...","Premier League, Ederson and AlissonLiverpool, ...","20m/£17.3, 20m/£17.3, 20, 50m/£43.4, 43.8, 60m...",top ten valuable Premier League goalkeeper rev...
1,"Marcus Rashford, Rashford, HTML5, Maintenance,...","Manchester United, Sun, United, Rolls-Royce, U...","700,000",England striker involved crash grandmother yes...
2,"Main NavigationSkip, Erik, Roy Hodgson, Erik, ...","United, United, BST, Sky Sports, Newcastle Uni...",NaN,Skip Main NavigationSkip contentHow follow Uni...
3,"Erik, Nerazzurri, David De Gea, Man Utd, Jan O...","Manchester United, LaLiga, Old Trafford, Inter...","43.8, 47.8, 50, 50, 43.6",Manchester United reportedly weighing January ...
4,"Mateo Kovacic, Kovacic, Kovacic, Mateo Kovacic...","City, Red Star Belgrade, the Champions League,...",£30 million,Pep Guardiola gave injury update Manchester Ci...


In [ ]:
all_feed_all_docs_clean.shape

(212853, 4)

In [ ]:
all_feed_all_docs_clean.to_csv('/content/drive/MyDrive/nlp/all_feed_all_docs_clean_token.csv', index=False)